In [1]:
from tqdm.notebook import tqdm

In [2]:
import tslearn
import pandas as pd
import cupy
import numpy
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re
from astropy.table import Table
from cuml.multiclass import OneVsRestClassifier
from cuml.svm import LinearSVC
from cuml.linear_model import LogisticRegression
from cuml.metrics import accuracy_score
from cuml.cluster import KMeans as KMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split


def read_sfh_binned_data(dirname,filename):
    result = re.search('binned_SFHs-(.*)levels-JWST_(.*).txt', filename)
    if result:
        levels = int(result.group(1))
        z_bins = result.group(2)
        IDs, SFH_lev = read_sfh_data(dirname,filename,levels)
        return IDs, SFH_lev, levels, z_bins
    else:
        print("Filename doesn't contain levels or z_bins")

def read_sfh_data(dirname,filename,levels):
    data = join(dirname, filename)
    if isfile(data):
        df = pd.read_csv(data,sep='\t')    
        levels=df.columns[2:levels+2]
        SFH_lev=df[levels].values
        IDs = df['id_L19'].astype(int)
        return IDs, SFH_lev
    
def run_kmeans(x,num_clusters):
    x = TimeSeriesScalerMeanVariance().fit_transform(x)        
    X_train=cupy.asarray(x)
    kmeans = KMeans(n_clusters=num_clusters, max_iter=300, init='scalable-k-means++')
    kmeans.fit(X_train)
    cluster = cupy.asnumpy(kmeans.fit_predict(X_train))
    cluster_centers = cupy.asnumpy(kmeans.cluster_centers_)
    return x, cluster, cluster_centers

def get_sfh_cluster(dirname,filename,num_clusters):
    df_cluster = pd.DataFrame()
    IDs, SFH_lev, _, _ = read_sfh_binned_data(dirname,filename)
    _, cluster, _ = run_kmeans(SFH_lev,num_clusters)
    df_cluster['ID'] = IDs
    df_cluster['cluster'] = cluster
    return df_cluster

def get_photometry(filepath):
    data=Table.read(filepath)
    df_photometry=data.to_pandas()
    return df_photometry
    
def get_flux_cluster(df_cluster,df_photometry):
    df_photometry = df_photometry.merge(df_cluster, on= 'ID')
    df_flux = df_photometry[['ID','u_FLUX', 'gHSC_FLUX', 'rHSC_FLUX', 'iHSC_FLUX', 'zHSC_FLUX', 'cluster']]
    return df_flux
    
def train_classifier(dataset, classes):
    features = dataset[["u_FLUX","gHSC_FLUX","rHSC_FLUX","iHSC_FLUX","zHSC_FLUX"]]
    labels = dataset["cluster"]
    X_data = features.to_numpy()
    Y_data= labels.to_numpy()
    
    #Create a scaler model that is fit on the input data.
    scaler = StandardScaler().fit(X_data)

    #Scale the numeric feature variables
    X_data = scaler.transform(X_data)

    #Convert target variable as a one-hot-encoding array
    Y_data = label_binarize(Y_data,classes=classes)
    
    #Split training and test data
    X_train,X_test,Y_train,Y_test = train_test_split( X_data, Y_data, train_size=0.75)

    X_test,X_val,Y_test,Y_val = train_test_split( X_test, Y_test, test_size=0.5)

    classifier = OneVsRestClassifier(LinearSVC(probability=True))
    #classifier = OneVsRestClassifier(LogisticRegression())
    classifier.fit(X_train, Y_train)
    Y_test_hat = classifier.predict(X_test)
    
    score = accuracy_score(Y_test, Y_test_hat)

    return classifier, score

def run_pipeline(SFH_dirname,SFH_filename,FITS_filepath,num_clusters=6):
    df_photometry = get_photometry(FITS_filepath)
    df_cluster = get_sfh_cluster(SFH_dirname,SFH_filename,num_clusters)
    df_flux = get_flux_cluster(df_cluster,df_photometry)
    classes = []
    for i in range(num_clusters):
        classes.append(i)
    classifier, score = train_classifier(df_flux, classes)
    return classifier, score

/gpfswork/rech/owt/commun/galaxy_classification/.local_rapids/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [3]:
myDir = "/gpfswork/rech/owt/commun/galaxy_classification/2023-sfh-galaxy-classification/data/binned_SFHs/"
mySFH = 'binned_SFHs-11levels-JWST_z_0.5-1.0.txt'
myFITS = '/gpfswork/rech/owt/commun/galaxy_classification/2023-sfh-galaxy-classification/data/Horizon_AGN-COSMOS_like/HorizonAGN_COSMOS-Web_v2.0_witherr.fits'


## Plots

### 1. Extract labels from Input Data

Get the files list and the following labels:
1. SFH binning resolution (levels)
2. Redshift binning (z_start-end)

In [ ]:
myFiles = sorted(listdir(myDir))
files = [f for f in myFiles if isfile(join(myDir, f))]
z_bins_list = []
level_bins_list = []
for data in files:
    result = re.search('binned_SFHs-(.*)levels-JWST_(.*).txt', data)
    level_bins = int(result.group(1))
    z_bins = result.group(2)
    z_bins_list.append(z_bins)
    level_bins_list.append(level_bins)
    
z_bins_list=sorted(list(set(z_bins_list)))
level_bins_list=sorted(list(set(level_bins_list)))
print(level_bins_list, z_bins_list)

[7, 8, 9, 10, 11] ['z_0.5-1.0', 'z_1.0-1.5', 'z_1.5-2.0', 'z_2.0-2.5', 'z_2.5-3.0', 'z_3.0-3.5', 'z_3.5-4.0']


### 2. Plot cluster centers for each file

Pipeline:
1. Preprocessing unsing TimeSeriesScalerMeanVariance from tslearn
2. Clustering using Kmeans from cuml
3. Plot cluster centers

In [ ]:
outDir = "cluster_centers"

num_clusters = 6
seed = 0
numpy.random.seed(seed)

for data in tqdm(files):
    _, x, level_bins, z_bins = read_sfh_binned_data(myDir, data)    
    x, y_pred, cluster_centers = run_kmeans(x,num_clusters)                                        
    fig = plt.figure(figsize=(12, 18), dpi=150)
    
    for yi in range(6):
        plt.subplot(6, 3, yi + 1)
        plt.plot(cluster_centers[yi].ravel(), "r-")
        plt.xlim(0, x.shape[1])
        plt.ylim(-4, 4)
        plt.text(0.55, 0.85, 'Cluster %d' % (yi + 1), transform=plt.gca().transAxes)
        if yi == 1:
            plt.title(str(level_bins)+"levels "+z_bins)

    plt.tight_layout()    
    fig.savefig(join(outDir,data+'.png'))
    plt.close()

  0%|          | 0/35 [00:00<?, ?it/s]

### 3. Plot cluster centers for each SFH binning resolution

Pipeline:
1. Preprocessing unsing TimeSeriesScalerMeanVariance from tslearn
2. Clustering using Kmeans from cuml
3. Plot cluster centers

In [ ]:
outDir = "cluster_centers_2"

num_clusters = 6
seed = 0
numpy.random.seed(seed)
pre_level_bins = int(10)
plt_id = 0                                         
fig = plt.figure(figsize=(12, 18), dpi=150)

for data in tqdm(files):
    _, x, level_bins, z_bins = read_sfh_binned_data(myDir, data)    
    x, y_pred, cluster_centers = run_kmeans(x,num_clusters)
    
    if pre_level_bins != level_bins:
        plt.tight_layout()
        #plt.show()
        fig.savefig(join(outDir,str(pre_level_bins)+'levels.png'))
        plt.close()
        plt_id = 0    
        fig = plt.figure(figsize=(12, 18), dpi=150)
        
    plt_id = plt_id + 1
    plt.subplot(8,4,plt_id)
    for yi in range(6):
        plt.plot(cluster_centers[yi].ravel(),label='Cluster centers %d' % (yi + 1))    
    plt.text(0.55, 0.85,z_bins, transform=plt.gca().transAxes)
    plt.xlim(0, x.shape[1])
    plt.ylim(-4, 4)
    if plt_id == 1:
        plt.title('Cluster centers '+str(level_bins)+'levels')
    
    pre_level_bins = level_bins

plt.tight_layout()
#plt.show()
fig.savefig(join(outDir,str(pre_level_bins)+'levels.png'))
plt.close()

### 4. Plot cluster centers for each redshift bins

Pipeline:
1. Preprocessing unsing TimeSeriesScalerMeanVariance from tslearn
2. Clustering using Kmeans from cuml
3. Plot cluster centers

In [ ]:
outDir = "cluster_centers_3"

num_clusters = 6
seed = 0
numpy.random.seed(seed)
pre_z_bins = "z_0.5-1.0"
plt_id = 0    
fig = plt.figure(figsize=(12, 18), dpi=150)

for z_bins in tqdm(z_bins_list):
    for level in range(7,12):
        data = 'binned_SFHs-'+str(level)+'levels-JWST_'+z_bins+'.txt'
        _, x, level_bins, z_bins = read_sfh_binned_data(myDir, data)    
        x, y_pred, cluster_centers = run_kmeans(x,num_clusters)   

        if pre_z_bins != z_bins:
            plt.tight_layout()
            #plt.show()
            fig.savefig(join(outDir,pre_z_bins+'.png'))
            plt.close()
            plt_id = 0    
            fig = plt.figure(figsize=(12, 18), dpi=150)

        plt_id = plt_id + 1
        plt.subplot(6,3,plt_id)
        for yi in range(6):
            plt.plot(cluster_centers[yi].ravel(),label='Cluster centers %d' % (yi + 1))    
        plt.text(0.55, 0.85,str(level)+'levels', transform=plt.gca().transAxes)
        plt.xlim(0, x.shape[1])
        plt.ylim(-4, 4)
        if plt_id == 1:
            plt.title('Cluster centers '+z_bins)

        pre_z_bins = z_bins
                
plt.tight_layout()
#plt.show()
fig.savefig(join(outDir,pre_z_bins+'.png'))
plt.close()

### 5. Plot cluster histogram for each SFH binning resolution

Pipeline:
1. Preprocessing unsing TimeSeriesScalerMeanVariance from tslearn
2. Clustering using Kmeans from cuml
3. Plot cluster histogram

In [ ]:
outDir = "hist_2"

num_clusters = 6
seed = 0
numpy.random.seed(seed)
pre_level_bins = int(10)
plt_id = 0    
fig = plt.figure(figsize=(12, 18), dpi=150)

for data in tqdm(files):
    _, x, level_bins, z_bins = read_sfh_binned_data(myDir, data)    
    x, y_pred, cluster_centers = run_kmeans(x,num_clusters)       
    
    if pre_level_bins != level_bins:
        plt.tight_layout()
        #plt.show()
        fig.savefig(join(outDir,str(pre_level_bins)+'levels.png'))
        plt.close()
        plt_id = 0    
        fig = plt.figure(figsize=(12, 18), dpi=150)        
    
    plt_id = plt_id + 1
    plt.subplot(8,4,plt_id)
    plt.hist(y_pred)   
    plt.text(0.55, 0.85,z_bins, transform=plt.gca().transAxes)
    if plt_id == 1:
        plt.title('Cluster histogram '+str(level_bins)+'levels')
        
    pre_level_bins = level_bins

plt.tight_layout()
#plt.show()
fig.savefig(join(outDir,str(pre_level_bins)+'levels.png'))
plt.close()

### 6. Plot cluster histogram for each redshift bins

Pipeline:
1. Preprocessing unsing TimeSeriesScalerMeanVariance from tslearn
2. Clustering using Kmeans from cuml
3. Plot cluster histogram

In [ ]:
outDir = "hist_3"

num_clusters = 6
seed = 0
numpy.random.seed(seed)
pre_z_bins = "z_0.5-1.0"
plt_id = 0    
fig = plt.figure(figsize=(12, 18), dpi=150)

for z_bins in tqdm(z_bins_list):
    for level in range(7,12):
        data = 'binned_SFHs-'+str(level)+'levels-JWST_'+z_bins+'.txt'
        _, x, level_bins, z_bins = read_sfh_binned_data(myDir, data)    
        x, y_pred, cluster_centers = run_kmeans(x,num_clusters)

        if pre_z_bins != z_bins:
            plt.tight_layout()
            #plt.show()
            fig.savefig(join(outDir,pre_z_bins+'.png'))
            plt.close()
            plt_id = 0    
            fig = plt.figure(figsize=(12, 18), dpi=150)

        plt_id = plt_id + 1
        plt.subplot(6,3,plt_id)
        plt.hist(y_pred)   
        plt.text(0.55, 0.85,str(level)+'levels', transform=plt.gca().transAxes)
        if plt_id == 1:
            plt.title('Cluster histogram '+z_bins)

        pre_z_bins = z_bins
                
plt.tight_layout()
#plt.show()
fig.savefig(join(outDir,pre_z_bins+'.png'))
plt.close()

### 7. Plot cluster centers and histogram for each redshift bins at a given SFH resolution

Pipeline:
1. Preprocessing unsing TimeSeriesScalerMeanVariance from tslearn
2. Clustering using Kmeans from cuml
3. Plot cluster centers and histogram

In [ ]:
outDir = "cluster_centers_hist"

num_clusters = 6
seed = 0
numpy.random.seed(seed)
plt_id = 0    
fig = plt.figure(figsize=(12, 18), dpi=150)

for z_bins in tqdm(z_bins_list):
    data = 'binned_SFHs-11levels-JWST_'+z_bins+'.txt'
    _, x, level_bins, z_bins = read_sfh_binned_data(myDir, data)    
    x, y_pred, cluster_centers = run_kmeans(x,num_clusters)

    plt_id = plt_id + 1
    plt.subplot(14,7,plt_id)
    for yi in range(6):
        plt.plot(cluster_centers[yi].ravel(),label='Cluster centers %d' % (yi + 1))    
    plt.text(0.55, 0.85,z_bins, transform=plt.gca().transAxes)
    plt.xlim(0, x.shape[1])
    plt.ylim(-4, 4)

    plt.subplot(14,7,plt_id+7)
    plt.hist(y_pred)   
    plt.text(0.55, 0.85,z_bins, transform=plt.gca().transAxes)
                
plt.tight_layout()
#plt.show()
fig.savefig(join(outDir,'11levels.png'))
plt.close()

## Classifier

### Pipeline example of classifier

Pipeline:
1. Get fluxes and clusters for different number of clusters
2. Prepare the dataset
3. Train classifier

#### 1. Get fluxes and clusters for different number of clusters

In [4]:
df_photometry = get_photometry(myFITS)

In [7]:
num_clusters = 6
classes = []
for i in range(num_clusters):
    classes.append(i)
df_cluster = get_sfh_cluster(myDir,mySFH,num_clusters)
df_flux = get_flux_cluster(df_cluster,df_photometry)
    

#### 2. Prepare the dataset
1. Load data into a pandas dataframe
2. Convert the dataframe to a numpy array
3. Scale the feature dataset
4. Use one-hot-encoding for the target variable
5. Split into training and test datasets

In [8]:
dataset = df_flux
features = dataset[["u_FLUX","gHSC_FLUX","rHSC_FLUX","iHSC_FLUX","zHSC_FLUX"]]
labels = dataset["cluster"]
X_data = features.to_numpy()
Y_data= labels.to_numpy()

print("\nFeatures before scaling :\n------------------------------------")
print(X_data[:5,:])
print("\nTarget before scaling :\n------------------------------------")
print(Y_data[:5])

#Create a scaler model that is fit on the input data.
scaler = StandardScaler().fit(X_data)

#Scale the numeric feature variables
X_data = scaler.transform(X_data)

#Convert target variable as a one-hot-encoding array
Y_data = label_binarize(Y_data,classes=classes[:i+2])

print("\nFeatures after scaling :\n------------------------------------")
print(X_data[:5,:])
print("\nTarget after one-hot-encoding :\n------------------------------------")
print(Y_data[:5])

#Split training and test data
X_train,X_test,Y_train,Y_test = train_test_split( X_data, Y_data, train_size=0.75)

X_test,X_val,Y_test,Y_val = train_test_split( X_test, Y_test, test_size=0.5)

print("\nTrain Test Val Dimensions:\n------------------------------------")
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape, X_val.shape, Y_val.shape)



Features before scaling :
------------------------------------
[[0.28672666 0.32038249 0.50008666 0.96038686 1.64027649]
 [0.20722867 0.24348071 0.36668962 0.7010275  1.13553877]
 [0.16796237 0.14784522 0.15487576 0.30383067 0.45300291]
 [0.36409947 0.385252   0.58739122 1.11396946 1.8537296 ]
 [0.03021147 0.04522976 0.05992277 0.13502131 0.23168474]]

Target before scaling :
------------------------------------
[1 3 0 2 1]

Features after scaling :
------------------------------------
[[-0.24104421 -0.31081349 -0.32901012 -0.3455134  -0.28738264]
 [-0.39089942 -0.40103601 -0.39092929 -0.40155184 -0.36778537]
 [-0.46491714 -0.51323723 -0.48924736 -0.48737211 -0.47651065]
 [-0.095195   -0.23470744 -0.28848578 -0.3123296  -0.2533804 ]
 [-0.72458018 -0.63362749 -0.53332188 -0.52384587 -0.51176576]]

Target after one-hot-encoding :
------------------------------------
[[0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]]

Train Test Val Dimensions:
------------------

#### 3. Train classifier

In [9]:
classifier = OneVsRestClassifier(LinearSVC(probability=True))
#classifier = OneVsRestClassifier(LogisticRegression())
classifier.fit(X_train, Y_train)
Y_test_hat = classifier.predict(X_test)

print('Accuracy:', accuracy_score(Y_test, Y_test_hat))

Accuracy: 0.875317394733429


### Run pipeline with different cluster numbers

In [10]:
for num_clusters in tqdm(range(2,11)):
    classifier, score = run_pipeline(myDir,mySFH,myFITS,num_clusters)
    print(score)

  0%|          | 0/9 [00:00<?, ?it/s]

0.7165747284889221
0.7734874486923218
0.8235706090927124
0.8405568599700928
0.87724369764328
0.8616583347320557
0.8920409679412842
0.9043866395950317
0.9125295281410217
